In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
data = pd.read_table('taxid10239.nbr', comment='#', header=None, names=["Representative","Neighbor","Host","Selected lineage","Taxonomy name","Segment name"])

In [ ]:
data

In [49]:
accession = list(set(data['Neighbor']))
print(len(accession))

accession.pop(235127)
print(len(accession))


248779
248778


In [64]:
def check(segment):
    with open('/home/bio_kang/data/virus/download.txt','r') as f: # overlop write
        downloads = f.read().strip('\n').split('\n')
        if set(downloads) > set(segment):
            return "False"
        else:
            with open('/home/bio_kang/data/virus/download.txt', 'r+') as k:
                url_add = k.read().strip('\n').split('\n')
                url_add = list(set(downloads))
                url_add = url_add + segment
            with open('/home/bio_kang/data/virus/download.txt', 'w') as j:
                j.write("\n".join(url_add))
            return "True" 

function format

In [ ]:
# Complete code

import pandas as pd
from bs4 import BeautifulSoup
import requests
import re


# the function is designed to get fasta format information
def get_url_info(segment):

    # Using requests function get the id from term
    term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=f7aed0f55f83d46ff0249e3e1830661bef08&term=' + ",".join(segment)
    term_info = requests.get(term_url).text
    soup = BeautifulSoup(term_info, 'html.parser')

    # get id
    accession_id = [k.text for k in soup.find_all('id')]

    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=f7aed0f55f83d46ff0249e3e1830661bef08'.format(','.join(accession_id))
    fasta_info = requests.get(fasta_url).text
    return fasta_info

# this function is design to check downloading 
def check(segment):
    with open('/home/bio_kang/data/virus/download.txt','r') as f: # overlop write
        downloads = f.read().strip('\n').split('\n')
        if set(downloads) > set(segment):
            return "False"
        else:
            with open('/home/bio_kang/data/virus/download.txt', 'r+') as k:
                url_add = k.read().strip('\n').split('\n')
                url_add = list(set(downloads))
                url_add = url_add + segment
            with open('/home/bio_kang/data/virus/download.txt', 'w') as j:
                j.write("\n".join(url_add))
            return "True" 


# this function is designed to write files
def write_file(fasta_info,name_num):
    with open("/home/bio_kang/data/virus/sequence_{}.fasta".format(name_num), 'w') as f:
        f.write(fasta_info)


# read information from taxid10239.nbr file
data = pd.read_table('taxid10239.nbr', comment='#', header=None, names=["Representative","Neighbor","Host","Selected lineage","Taxonomy name","Segment name"])

# extract 'Neighbor' information
accession = list(set(data['Neighbor']))



for i in range(1,12440):
    segment = accession[(i-1)*20:i*20]
    try:
        if check(segment) == 'True':
            fasta_info = get_url_info(segment)
            write_file(fasta_info, i)
        else:
            continue
    except:
        continue

In [67]:
len(accession)

248778

In [ ]:
# Complete code

import pandas as pd
from bs4 import BeautifulSoup
import requests

data = pd.read_table('/home/bio_kang/Learning/Python/requests/taxid10239.nbr', comment='#', header=None, names=["Representative","Neighbor","Host","Selected lineage","Taxonomy name","Segment name"]) # get information

accession = list(data['Neighbor']) # extract 'Neighbor' information

# Becauce the RetMax is 20, I slices their length with 20 and the last length is 13
for i in range(13050):
    segment = accession[i*20:(i+1)*20]

    # Using requests function get the id of term
    term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=f7aed0f55f83d46ff0249e3e1830661bef08&term=' + ",".join(segment)
    term_info = requests.get(term_url).text
    soup = BeautifulSoup(term_info, 'html.parser')

    accession_id = [k.text for k in soup.find_all('id')] # get id

    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=f7aed0f55f83d46ff0249e3e1830661bef08'.format(','.join(accession_id))
    fasta_info = requests.get(fasta_url).text

    # get fasta file
    with open("/home/bio_kang/Learning/Python/requests/data/sequence_{}.fasta".format(i), 'w') as f:
        f.write(fasta_info)


check processes

In [ ]:
renew = []
with open('/home/bio_kang/data/virus_data/complement_data/omitted') as f:
    omitted_info_set = f.read().split('\n')
    omitted_info_set.pop(-1)
for omitted_info in omitted_info_set:
    pattern = re.compile(r'\d+')
    renew = renew + re.findall(pattern,omitted_info)

omitted = []
for num in renew:
    omitted = omitted + accession[(int(num)-1)*20:int(num)*20]

omitted.pop(25) # delete NCBI_GENOMES

for num in range(20):
    term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=bb9bbede507a3504dc8923c13b4e94080c08&term=' + ','.join(omitted[num*20: (num+1)*20])
    term_info = requests.get(term_url).text
    soup = BeautifulSoup(term_info, 'html.parser')

    accession_id = [k.text for k in soup.find_all('id')]

    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=bb9bbede507a3504dc8923c13b4e94080c08'.format(','.join(accession_id))
    fasta_info = requests.get(fasta_url).text

    with open('/home/bio_kang/data/virus_data/complement_data/complement_{}.fasta'.format(num), 'w') as f:
        f.write(fasta_info)

In [81]:
seg = 'EF537008'
term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=bb9bbede507a3504dc8923c13b4e94080c08&term=' + seg
term_info = requests.get(term_url).text
soup = BeautifulSoup(term_info, 'html.parser')
id = soup.find_all('id')
print(id)

[<id>146230032</id>]


In [47]:
import re
pattern = re.compile(r'[a-z]{8}_\d+.\w{5}')
ls = re.search(pattern,'/home/bio_kang/data/virus_data/sequence_3307.fasta')
print(ls)

<re.Match object; span=(31, 50), match='sequence_3307.fasta'>


In [70]:
term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key=bb9bbede507a3504dc8923c13b4e94080c08&term=' + "NCBI_GENOMES"
term_info = requests.get(term_url).text
pattern = re.compile(r'\d{10}')
id = re.findall(pattern, term_info)
fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=bb9bbede507a3504dc8923c13b4e94080c08'.format(','.join(id))
fasta_info = requests.get(fasta_url).text
with open('/home/bio_kang/data/virus_data/complement_data/NCBI_GENOMES', 'w') as f:
    f.write(fasta_info)

In [5]:
with open('/home/bio_kang/data/virus_data/complement_data/omitted_num', 'r') as f:
    omitted_info = f.read().strip('\n').split('\n')
    print(omitted_info)

['10884', '11093', '11185', '11445', '11561', '11562', '12163', '12252', '12456', '12475', '12569', '12572', '12607', '12833', '12850', '12913', '12974', '12996', '13001', '13004', '3210', '3307', '5224', '5851']


In [40]:
import re
for num in omitted_info:
    with open('/home/bio_kang/data/virus_data/sequence_{}.fasta'.format(num), 'r') as f:
        down_info = f.read()
    pattern = re.compile(r'>(\w{1,2}\d+)')
    down_term = re.findall(pattern, down_info)
    num = int(num)
    print(set(accession[(num-1)*20: num*20]) - set(down_term))


set()
{'EF065516', 'MF618252', 'HM211098', 'MT337384', 'MF618253', 'HM211101', 'AF207902', 'FJ647221', 'KF268336', 'FJ884687', 'EF065515', 'KP887098', 'MG762674', 'EF065513', 'HM211100', 'FJ647218', 'JF792617', 'NCBI_GENOMES', 'FJ647225', 'EF065514'}
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
{'KF550407', 'MF795136', 'MN059372', 'MN059348', 'MN059368', 'MN059370', 'MN059347', 'KR819505', 'MN059386', 'MN059369', 'MN059343', 'MN059366', 'MN059357', 'MN059338', 'MN059350', 'MT731500', 'MN059355', 'MN059351', 'MN059362', 'MN059384'}
{'KM607358', 'KM607428', 'KM607395', 'KX979912', 'KM607324', 'KM607438', 'MT433357', 'KM607296', 'KM607318', 'KM607343', 'KM607411', 'KM607295', 'KM607310', 'KM607394', 'KM607435', 'KM607386', 'L41577', 'KU759876', 'KY434064', 'KM607317'}
{'KR705240', 'KT920921', 'KP188803', 'KJ753540', 'FJ206205', 'MG181937', 'EF554099', 'MG181750', 'JX040426', 'MF161622', 'KT919244', 'KM434193', 'KJ627144', 'KJ4

In [ ]:
accession[11093*20: 11094*20]

In [16]:
url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&term={}'.format(','.join(accession[11093*20:11094*20]))
print(url)

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&term=HM211099,MH002339,EF065505,EF065508,MH002337,EF065506,EF065507,DQ648794,MW218395,MH002338,KJ473822,MH002340,EF065509,EF065510,MH002342,KJ473820,EF065512,EF065511,MH002341,AY545914


In [27]:
train = accession[10883*20:10884*20]
print(train)
down_train = ['EF537008', 'AB300379', 'LC432343', 'LC573905', 'LC573907', 'LC573906', 'KF861773', 'KF861768', 'KF861767', 'KF861769', 'KJ830761', 'MW848534', 'KJ830760', 'EF589670', 'KJ544560', 'MN059098', 'AJ316085', 'AB304848']





['EF537008', 'AB300379', 'LC432343', 'LC573905', 'LC573907', 'LC573906', 'KF861773', 'KF861768', 'KF861767', 'KF861769', 'KJ830761', 'MW848534', 'KJ830760', 'EF589670', 'KJ544560', 'MN059098', 'AJ316085', 'AB304848', 'KJ544560', 'MN059098']
[]
